# Equal-Weight S&P 500 Index Fund
## Introduction
The S&P 500 is the world's most popular stock market index.
The largest fund that is benchmarked to this index is the SPDR S&P 500 ETF Trust (SPY).

Goal: Give value of a portfolio, tell how many shares of each S&P 500 stock to buy to get an equal-weight version of the S&P 500.
### "More explainations to come.."

## Library Imports

In [6]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

from secret import IEX_CLOUD_API_TOKEN

## Importing Data

In [7]:
stock_tickers = pd.read_csv('Data/sp_500_stocks.csv')
stock_tickers

,Ticker
0,MSFT
1,AAPL
2,NVDA
3,GOOG
4,GOOGL
...,...
498,IVZ
499,GNRC
500,CMA
501,XRAY


## API Calls

**Quotes**: Market Capitalization and Price

In [8]:
symbol = 'AAPL'
api_base_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
get_response = requests.get(api_base_url)
data = get_response.json()
data

{'avgTotalVolume': 73817592,
 'calculationPrice': 'close',
 'change': 0.91,
 'changePercent': 0.00531,
 'close': 172.28,
 'closeSource': 'official',
 'closeTime': 1711137600319,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': 172.29,
 'delayedPriceTime': 1711137586340,
 'extendedChange': 0.02,
 'extendedChangePercent': 0.00012,
 'extendedPrice': 172.3,
 'extendedPriceTime': 1711151999309,
 'high': 173.05,
 'highSource': '15 minute delayed price',
 'highTime': 1711137599994,
 'iexAskPrice': 0,
 'iexAskSize': 0,
 'iexBidPrice': 0,
 'iexBidSize': 0,
 'iexClose': 172.27,
 'iexCloseTime': 1711137598471,
 'iexLastUpdated': 1711137598471,
 'iexMarketPercent': 0.013333532321143053,
 'iexOpen': 171.8,
 'iexOpenTime': 1711114200855,
 'iexRealtimePrice': 172.27,
 'iexRealtimeSize': 3,
 'iexVolume': 948816,
 'lastTradeTime': 1711137599913,
 'latestPrice': 172.28,
 'latestSource': 'Close',
 'latestTime': 'March 22, 2024',
 'latestUpdate': 1711137600319,
 'latestVolume': 71160138,


In [9]:
price = data['latestPrice']
market_cap = data['marketCap']

In [10]:
Dataframe_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
Dataframe = pd.DataFrame(columns=Dataframe_columns)
Dataframe

# Testing
Dataframe.append(
    pd.Series (
        [
            symbol,
            price, 
            market_cap,
            'N/A'
        ],
        index = Dataframe_columns
    ),
    ignore_index = True
)

C:\Users\gupta\AppData\Local\Temp\ipykernel_31580\268769306.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Dataframe.append(


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,172.28,2660327258680,N/A


**Single Fetch**

In [11]:
# Dataframe = pd.DataFrame(columns = Dataframe_columns)
# for stock_ticker in stock_tickers["Ticker"]:
#     api_url = f'https://cloud.iexapis.com/stable/stock/{stock_ticker}/quote/?token={IEX_CLOUD_API_TOKEN}'
#     stock_data = requests.get(api_url).json() 
#     Dataframe = Dataframe.append(
#         pd.Series (
#             [
#                 stock_ticker,
#                 stock_data['latestPrice'], 
#                 stock_data['marketCap'],
#                 'N/A'
#             ],
#             index = Dataframe_columns
#         ),
#         ignore_index = True
#     )

# Dataframe

**Batch Fetch**

In [12]:
def chunks(series, n):
    for i in range(0, len(series), n):
        yield series[i:i + n]

symbol_sublists = list(chunks(stock_tickers['Ticker'], 100))
symbol_sublists_tickers = []
for i in range(0, len(symbol_sublists)):
    symbol_sublists_tickers.append(','.join(symbol_sublists[i]))

Dataframe = pd.DataFrame(columns = Dataframe_columns)
for symbol in symbol_sublists_tickers:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    stock_data = requests.get(batch_api_call_url).json()
    for stock_ticker in symbol.split(','):
        Dataframe = Dataframe.append(
            pd.Series (
                [
                    stock_ticker,
                    stock_data[stock_ticker]['quote']['latestPrice'], 
                    stock_data[stock_ticker]['quote']['marketCap'],
                    'N/A'
                ],
                index = Dataframe_columns
            ),
            ignore_index = True
        )

Dataframe

C:\Users\gupta\AppData\Local\Temp\ipykernel_31580\1301190705.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Dataframe = Dataframe.append(
C:\Users\gupta\AppData\Local\Temp\ipykernel_31580\1301190705.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Dataframe = Dataframe.append(
C:\Users\gupta\AppData\Local\Temp\ipykernel_31580\1301190705.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Dataframe = Dataframe.append(
C:\Users\gupta\AppData\Local\Temp\ipykernel_31580\1301190705.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Dataframe = Dataframe.append(
C:\Users\gupta\AppData\Local\Temp\ipykernel_31580\1301190705.py:15: 

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,MSFT,428.74,3185725228821,N/A
1,AAPL,172.28,2660327258680,N/A
2,NVDA,942.89,2357225000000,N/A
3,GOOG,151.77,1752944530063,N/A
4,GOOGL,150.77,1901058930000,N/A
...,...,...,...,...
498,IVZ,15.71,7056999050,N/A
499,GNRC,115.91,6985815722,N/A
500,CMA,51.65,6843091301,N/A
501,XRAY,32.75,6791147289,N/A


In [13]:
portfolio = input('Enter the value of your portfolio: ')
try:
    val = float(portfolio)
except ValueError:
    print('Please enter a number')
    portfolio = input('Enter the value of your portfolio: ')
    val = float(portfolio)

In [14]:
position_size = val / len(Dataframe.index)
for i in range(0, len(Dataframe.index)):
    Dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / Dataframe.loc[i, 'Stock Price'])

Dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,MSFT,428.74,3185725228821,4
1,AAPL,172.28,2660327258680,11
2,NVDA,942.89,2357225000000,2
3,GOOG,151.77,1752944530063,13
4,GOOGL,150.77,1901058930000,13
...,...,...,...,...
498,IVZ,15.71,7056999050,126
499,GNRC,115.91,6985815722,17
500,CMA,51.65,6843091301,38
501,XRAY,32.75,6791147289,60


## XLSX File Output

In [15]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
Dataframe.to_excel(writer, 'Recommended Trades', index = False)

In [16]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of Shares to Buy', integer_format]
}

In [17]:
writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
writer.sheets['Recommended Trades'].write('B1', 'Stock Price', dollar_format)
writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization', dollar_format)
writer.sheets['Recommended Trades'].write('D1', 'Number of Shares to Buy', integer_format)
writer.save()

In [18]:
column_format = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of Shares to Buy', integer_format]
}

for column in column_format.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_format[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_format[column][0], column_format[column][1])

In [19]:
writer.save()

c:\Users\gupta\AppData\Local\Programs\Python\Python310\lib\site-packages\xlsxwriter\workbook.py:369: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
